# Building a Chatbot
## Part III: Building a chatbot UI
Part I and II of exercise 4 taught us the mechanics of instructing the chatbot to answer user queries based on a custom databse that we define.

An important part of any machine learning project is how you share it with other people. We spent all our time executing code snippets in this workshop. For our final notebook, we will look at how [Streamlit](https://streamlit.io/) can help us build a good looking user interface for our chatbot. As expected, we need to bring the modules that we created in parts I and II and organize them.


##Libraries

In [ ]:
%%capture
!pip install rank_bm25 pypdf2 tiktoken openai

## Streamlit
Streamlit requires all your python script inside a py file. The first line is a magic command that will write all the instructions we have had before into a single `chatbot.py` file. Let's quickly review the code together.

In [ ]:
%%writefile chatbot.py
# load libraries
import time
import streamlit as st
import pickle
import os
import json
import requests

# """
#   ____  _               _
#  / ___|| |_ ___ _ __   / |
#  \___ \| __/ _ \ '_ \  | |
#   ___) | ||  __/ |_) | | |
#  |____/ \__\___| .__/  |_|
#                |_|
# We define functions that load processed data and models into cache using streamlit's caching mechanism.
# The cache command make the streamlit app faster and more responsive.
# """
# define functions for chat gpt

# POPULATE THESE PARAMETERS BEFORE RUNNING
#API_KEY = 
#ASSISTANT_ID = 
#THREAD_ID =

#access_code = 
#project_id = 

#these are all the fucntions for Custom GPT
BASE_URL = "https://api.openai.com/v1"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v2"
}

def add_message(thread_id, content):
    message_data = {
        "role": "user",
        "content": content
    }
    requests.post(
        f"{BASE_URL}/threads/{thread_id}/messages",
        headers=HEADERS,
        json=message_data
    )

def run_assistant(thread_id, assistant_id):
    run_data = {"assistant_id": assistant_id}
    response = requests.post(
        f"{BASE_URL}/threads/{thread_id}/runs",
        headers=HEADERS,
        json=run_data
    )
    return response.json()['id']

def wait_for_run_completion(thread_id, run_id):
    while True:
        response = requests.get(
            f"{BASE_URL}/threads/{thread_id}/runs/{run_id}",
            headers=HEADERS
        )
        status = response.json()['status']
        if status == 'completed':
            break
        time.sleep(0.5)  # Poll every 0.5 seconds
        # return response


def get_assistant_reply(thread_id):
    response = requests.get(
        f"{BASE_URL}/threads/{thread_id}/messages",
        headers=HEADERS
    )
    return response.json()['data'][0]['content'][0]['text']['value']


def get_completion_from_messages(messages):
    """This function handles calls to the openai api and returns the response from the chatgpt model"""
    add_message(THREAD_ID, messages)
    run_id = run_assistant(THREAD_ID, ASSISTANT_ID)
    wait_for_run_completion(THREAD_ID, run_id)
    reply = get_assistant_reply(THREAD_ID)
    return reply


#these are all the functions for getting data from Forma
def get_proposals(access_code):
    url = f"https://developer.api.autodesk.com/forma/proposal/v1alpha/proposals?authcontext={project_id}&limit=100"
    payload = {}
    headers = {
    'Authorization': f'Bearer {access_code}'
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()

def get_elements(proposals, access_code):
    master_dict = {}
    proposal_count = 1
    for proposal in proposals["results"]:
        for building_index in range(len(proposal["children"])):
            if building_index != 0:
                continue
            else:
                building_urn = proposal["children"][building_index]["urn"]
                # print(building_urn)
                url = f"https://developer.api.autodesk.com/forma/element-service/v1alpha/elements/{building_urn}?authcontext={project_id}"

                payload = {}
                headers = {
                'Authorization': f'Bearer {access_code}'
                }
                response = requests.request("GET", url, headers=headers, data=payload)
                key = f"proposal {proposal_count}"
                master_dict[key] = response.json()
        proposal_count += 1

    return master_dict


# ref: https://docs.streamlit.io/knowledge-base/tutorials/build-conversational-apps
st.title("Forma Chatbot")

if st.button('stream data'):
  proposals = get_proposals(access_code)
  data_input = get_elements(proposals, access_code)
  st.write(data_input)

  add_message(THREAD_ID, f"{data_input}")
  run_id = run_assistant(THREAD_ID, ASSISTANT_ID)
  wait_for_run_completion(THREAD_ID, run_id)
  reply = get_assistant_reply(THREAD_ID)

  st.write(reply)



if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.chat_hist = []

for message in st.session_state.chat_hist:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("What is up?"):
    llm_prompt = prompt

    st.session_state.messages.append({"role": "user", "content": llm_prompt})
    st.session_state.chat_hist.append({"role": "user", "content": llm_prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"): # addd an avatar: ,avatar="TT_icon.png"
            message_placeholder = st.empty()
            full_response = ""
            stream = get_completion_from_messages(llm_prompt)
            full_response = st.write(stream)
    st.session_state.messages.append({"role": "assistant", "content": stream})
    st.session_state.chat_hist.append({'role':'assistant', 'content':stream})

# print references:
# add a collapsible section to show reference documents
# if len(st.session_state.chat_hist)>0:
#     with st.expander("References"):
#         st.markdown("Reference documents:")
#         for i,doc in enumerate(st.session_state.chat_hist[0]['ref_docs']):
#             st.write(f"Reference {i+1}")
#             st.write("-"*20)
#             st.write(doc)



Overwriting chatbot.py


##Building a local server
The following lines are required to run a streamlit app in google colab. If you run this notebook localy, you only need to open an Anaconda propmt and type:
`streamlit run chatbot.py` while changing your directory (`cd`) to the path of chatbot.py.

In [ ]:
!pip install -q streamlit
!npm install localtunnel


up to date, audited 23 packages in 807ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run chatbot.py &>/content/logs.txt &

This step is also needed because we are running the app on google colab. Please copy the endpoint ip and use it after clicking on the generated url.

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.75.86.74


Make sure secret_workshop.txt is uploaded to the files section of colab before running the next cell.

In [ ]:
!npx localtunnel --port 8501

your url is: https://yellow-otters-give.loca.lt
